
### **NLP Lab Assignment - 04**

---



**Aim :** Implement a sentiment classifier using LSTM Network. Use word embeddings Word2Vec to represent words as input to the model that is sentiment classifier. Display accuracy of your model using 4-fold cross validation on the selected dataset.

**Name :** Kiran Patil

**ID :** 211070904


In [ ]:
!pip install tensorflow-gpu

  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf
print(tf.__version__)

2.14.0


In [ ]:
import gensim
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
import pandas as pd


In [ ]:
df=pd.read_csv('/content/drive/MyDrive/VJTI/4th_year/SEM_07/NLP_LAB/211070904_EXP_04/train.csv')

In [ ]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [ ]:
df.shape

(20800, 5)

In [ ]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [ ]:
###Drop Nan Values
df=df.dropna()

In [ ]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df.text)
sequences = tokenizer.texts_to_sequences(df.text)
X = pad_sequences(sequences, maxlen=6)

In [ ]:
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df.label)
y = to_categorical(y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Train a Word2Vec model on your corpus
sentences = [text.split() for text in df.text]
word2vec_model = gensim.models.Word2Vec(sentences, vector_size=100, window=5, min_count=1, sg=0)

In [ ]:
# Create an embedding matrix
vocab_size = len(tokenizer.word_index) + 1
embedding_dim = 100
embedding_matrix = np.zeros((vocab_size, embedding_dim))
for word, i in tokenizer.word_index.items():
    if word in word2vec_model.wv:
        embedding_matrix[i] = word2vec_model.wv[word]

In [ ]:
# Build the LSTM model
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, weights=[embedding_matrix], input_length=6, trainable=False))
model.add(LSTM(units=128))
model.add(Dense(units=2, activation='softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [ ]:
# Train the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=2)

Epoch 1/10
7314/7314 [==============================] - 52s 7ms/step - loss: 0.3851 - accuracy: 0.8124 - val_loss: 0.3797 - val_accuracy: 0.8020
Epoch 2/10
7314/7314 [==============================] - 54s 7ms/step - loss: 0.3121 - accuracy: 0.8574 - val_loss: 0.3580 - val_accuracy: 0.8307
Epoch 3/10
7314/7314 [==============================] - 50s 7ms/step - loss: 0.2570 - accuracy: 0.8886 - val_loss: 0.3768 - val_accuracy: 0.8266
Epoch 4/10
7314/7314 [==============================] - 50s 7ms/step - loss: 0.1959 - accuracy: 0.9194 - val_loss: 0.3990 - val_accuracy: 0.8283
Epoch 5/10
7314/7314 [==============================] - 52s 7ms/step - loss: 0.1434 - accuracy: 0.9466 - val_loss: 0.4872 - val_accuracy: 0.8305
Epoch 6/10
7314/7314 [==============================] - 53s 7ms/step - loss: 0.0975 - accuracy: 0.9655 - val_loss: 0.5690 - val_accuracy: 0.8296
Epoch 7/10
7314/7314 [==============================] - 50s 7ms/step - loss: 0.0680 - accuracy: 0.9786 - val_loss: 0.6155 - val_ac

In [ ]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Loss: {loss}, Accuracy: {accuracy}')

115/115 [==============================] - 1s 5ms/step - loss: 0.7944 - accuracy: 0.8176
Loss: 0.7943636775016785, Accuracy: 0.8176100850105286
